# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_after.png" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [9]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****OLMA
LANGSMITH_API_KEY=****185b
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [10]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [11]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [12]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [18]:
SYSTEM_PROMPT = """You are a careful SQLite analyst. 
Never use SQLite shell commands like `.tables`, `.schema`, `.headers`, `.help`.
Always use valid SQL. For example, to list tables, do:

SELECT name FROM sqlite_master WHERE type='table';

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [19]:
import os
from langchain_openai import ChatOpenAI

# HF LLM wrapper
llm = ChatOpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
    model="moonshotai/Kimi-K2-Thinking:novita",
    temperature=0.0,                # deterministic
    top_p=0.1,                      # suppress long rambling
    # max_tokens=2000,                 # <-- force short answers
    # presence_penalty=0,
    # frequency_penalty=0,
    # Any custom HuggingFace router params:
    # extra_body={
        # "stop": ["Tool Calls:", "Thinking:", "<thinking>"],  # Stop thinking dumps
    # },
)

In [20]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [21]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?


================================== Ai Message ==================================

I'll help you find the total on your last invoice, Frank. Let me look up your information.
Tool Calls:
  execute_sql (execute_sql:0)
 Call ID: execute_sql:0
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================


In [22]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?


================================== Ai Message ==================================

I'll help you find the titles in the database. Let me first explore what tables are available.
Tool Calls:
  execute_sql (execute_sql:0)
 Call ID: execute_sql:0
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================

I can see several tables that might contain titles. Let me check the most likely ones - Album and Track tables:


## Add memory

In [23]:
from langgraph.checkpoint.memory import InMemorySaver

In [24]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [25]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================

I'll help you find the total on your last invoice, Frank. Let me look up your information.
Tool Calls:
  execute_sql (execute_sql:0)
 Call ID: execute_sql:0
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (execute_sql:1)
 Call ID: execute_sql:1
  Args:
    query: SELECT CustomerId, FirstName, LastName FROM Customer WHERE FirstName = 'Frank' AND LastName = 'Harris';
===============

In [26]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (execute_sql:3)
 Call ID: execute_sql:3
  Args:
    query: 
SELECT t.TrackId, t.Name AS Title
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
WHERE il.InvoiceId = 374
ORDER BY il.InvoiceLineId;
================================= Tool Message =================================
Name: execute_sql

[(1803, 'Holier Than Thou'), (1807, 'Through The Never'), (1811, 'My Friend Of Misery'), (1815, 'The Wait'), (1819, 'Blitzkrieg'), (1823, 'So What')]
================================== Ai Message ==================================

Here are the titles on your last invoice:

1. **Holier Than Thou**
2. **Through The Never**
3. **My Friend Of Misery**
4. **The Wait**
5. **Blitzkrieg**
6. **So What**


## Try your own queries
Now that there is memory, check the agents recall!

In [ ]:
question = "Your Question Here?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)